In [75]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




In [25]:
felv = pd.read_csv("Desktop/Pazmany_Ossz.csv")

In [26]:
felv.Kar.unique()

array(['BTK', 'ITK', 'JAK', 'VJK'], dtype=object)

In [76]:
from bokeh.plotting import figure, output_file, output_notebook,  show, gridplot
from bokeh.models import ColumnDataSource

from bokeh.plotting import figure, show
from bokeh.sampledata.iris import flowers
from bokeh.models import ColumnDataSource
#from bokehutils.axes import xaxis



ImportError: No module named 'bokehutils'

In [5]:
output_notebook()

BokehJS successfully loaded.

In [6]:
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

In [27]:
allamiak = felv[(felv.Tipus == "A")]
btk = ColumnDataSource(allamiak[(allamiak.Kar == "BTK")])
itk = ColumnDataSource(allamiak[(allamiak.Kar == "ITK")])
jak = ColumnDataSource(allamiak[(allamiak.Kar == "JAK")])
vjk = ColumnDataSource(allamiak[(allamiak.Kar == "VJK")])
btk

In [8]:
p = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=450, plot_height=450)
p.scatter("Ev", "Ossz_Jel",  source = btk, legend ="BTK", color = "red")
p.line("Ev", "Ossz_Jel",  source = btk, legend ="BTK", color = "red")

In [9]:
p2 = figure(title="Felvettek szama osszesen", tools=TOOLS, plot_width=450, plot_height=450)
p2.scatter("Ev", "Ossz_Felv",  source = btk, legend ="BTK", color = "red")
p2.line("Ev", "Ossz_Felv",  source = btk, legend ="BTK", color = "red")

In [10]:
s = gridplot([[p,p2]])
show(s)

In [11]:
p.scatter("Ev", "Ossz_Jel",  source = itk, legend ="ITK", color = "orange")
p.line("Ev", "Ossz_Jel",  source = itk, legend ="ITK", color = "orange")
p2.scatter("Ev", "Ossz_Felv",  source = itk, legend ="ITK", color = "orange")
p2.line("Ev", "Ossz_Felv",  source = itk, legend ="ITK", color = "orange")

p.scatter("Ev", "Ossz_Jel",  source = vjk, legend ="VJK", color = "green")
p.line("Ev", "Ossz_Jel",  source = vjk, legend ="VJK", color = "green")
p2.scatter("Ev", "Ossz_Felv",  source = vjk, legend ="VJK", color = "green")
p2.line("Ev", "Ossz_Felv",  source = vjk, legend ="VJK", color = "green")

p.scatter("Ev", "Ossz_Jel",  source = jak, legend ="JAK", color = "blue")
p.line("Ev", "Ossz_Jel",  source = jak, legend ="JAK", color = "blue")
p2.scatter("Ev", "Ossz_Felv",  source = jak, legend ="JAK", color = "blue")
p2.line("Ev", "Ossz_Felv",  source = jak, legend ="JAK", color = "blue")

In [12]:
s = gridplot([[p,p2]])
show(s)

In [13]:
p = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=450, plot_height=450)
p.scatter("Ev", "Elso_Jel",  source = btk, legend ="BTK", color = "red")
p.line("Ev", "Elso_Jel",  source = btk, legend ="BTK", color = "red")
p.scatter("Ev", "Elso_Jel",  source = itk, legend ="ITK", color = "orange")
p.line("Ev", "Elso_Jel",  source = itk, legend ="ITK", color = "orange")
p.scatter("Ev", "Elso_Jel",  source = vjk, legend ="VJK", color = "green")
p.line("Ev", "Elso_Jel",  source = vjk, legend ="VJK", color = "green")
p.scatter("Ev", "Elso_Jel",  source = jak, legend ="JAK", color = "blue")
p.line("Ev", "Elso_Jel",  source = jak, legend ="JAK", color = "blue")
s = gridplot([[p,p2]])
show(s)

In [14]:
p = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=350, plot_height=350)
p.scatter( "Elso_Jel",  "Ossz_Felv", source = btk, legend ="BTK", color = "red")
p.line("Elso_Jel",  "Ossz_Felv",  source = btk, legend ="BTK", color = "red")
p2 = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=350, plot_height=350)
p2.scatter("Elso_Jel",  "Ossz_Felv",  source = itk, legend ="ITK", color = "orange")
p2.line("Elso_Jel",  "Ossz_Felv",  source = itk, legend ="ITK", color = "orange")
p3 = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=350, plot_height=350)
p3.scatter("Elso_Jel",  "Ossz_Felv",  source = vjk, legend ="VJK", color = "green")
p3.line("Elso_Jel",  "Ossz_Felv",  source = vjk, legend ="VJK", color = "green")
p4 = figure(title="Jelentkezok szama osszesen", tools=TOOLS, plot_width=350, plot_height=350)
p4.scatter("Elso_Jel",  "Ossz_Felv",  source = jak, legend ="JAK", color = "blue")
p4.line("Elso_Jel",  "Ossz_Felv",  source = jak, legend ="JAK", color = "blue")
s = gridplot([[p,p2], [p3, p4]])
show(s)

In [15]:
## correlation 
#np.corrcoef(allamiak["Ossz_Jel", "Elso_Jel"])
cor = np.corrcoef(allamiak[["Ossz_Jel", "Elso_Jel", "Nappali_Jel", "Allami_Jel", "Ossz_Felv", "Nappali_Felv", "Allami_Felv"]])
#allamiak
#cor.size
cor = np.corrcoef(allamiak[["Ossz_Jel", "Elso_Jel", "Nappali_Jel", "Allami_Jel", "Ossz_Felv", "Nappali_Felv", "Allami_Felv"]].as_matrix().T)
cor
x = ["Ossz_Jel", "Elso_Jel", "Nappali_Jel", "Allami_Jel", "Ossz_Felv", "Nappali_Felv", "Allami_Felv"]
trans = np.transpose([np.tile(x, len(x)), np.repeat(x, len(x))])

tmp_df = pd.DataFrame(data = {"cor" : cor.flatten(), "A": trans.T.flatten()[0:49], "B": trans.T.flatten()[49:197]})
tmp_df

,A,B,cor
0,Ossz_Jel,Ossz_Jel,1.000000
1,Elso_Jel,Ossz_Jel,0.924913
2,Nappali_Jel,Ossz_Jel,0.958203
3,Allami_Jel,Ossz_Jel,0.978995
4,Ossz_Felv,Ossz_Jel,0.961071
5,Nappali_Felv,Ossz_Jel,0.886703
6,Allami_Felv,Ossz_Jel,0.811004
7,Ossz_Jel,Elso_Jel,0.924913
8,Elso_Jel,Elso_Jel,1.000000
9,Nappali_Jel,Elso_Jel,0.838897


In [16]:
corgram = ColumnDataSource(tmp_df)

TOOLS = "resize,hover,save,pan,box_zoom,wheel_zoom"
A = tmp_df.A
B = tmp_df.B
A.values.T.tolist()

['Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv',
 'Ossz_Jel',
 'Elso_Jel',
 'Nappali_Jel',
 'Allami_Jel',
 'Ossz_Felv',
 'Nappali_Felv',
 'Allami_Felv']

In [17]:

p = figure(title="Szuper corgram",
    x_range=x, y_range=x,
    x_axis_location="above", plot_width=500, plot_height=500,
    toolbar_location="left", tools=TOOLS)

colors = [
    "#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce",
    "#ddb7b1", "#cc7878", "#933b41", "#550b1d"
]

#values = colors[tmp_df.cor.values * 4 + 4]
c = np.ceil(tmp_df.cor.values * 4 + 4)
#values = [colors[int(i)] for i in c]
#colors[int(8.0)]
values = [colors[int(i)] for i in c]
values
corgram.add(data = values, name = "colors")
corgram.add(data = np.tile([1, 2, 3, 4, 5, 6, 7], len(x)), name = "x_range")
corgram.add(data = np.repeat([1, 2, 3, 4, 5, 6, 7], len(x)), name = "y_range")
#corgram["colors"] = values
p.rect(A, B, 1, 1, line_color=None)
#show(p)

p.rect("x_range",
          "y_range",
          1, 1,
          source = corgram,
          color="colors")

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

show(p)  



In [28]:
## felveteli pontszamok?
pont = pd.read_csv("Desktop/Ponthatarok.csv")
pont.Max_pontszam = pont.Max_pontszam / pont.Max_pont
pont.Min_pontszam = pont.Min_pontszam / pont.Max_pont
btk = pont[(pont.Kar == "BTK")]
itk = pont[(pont.Kar == "ITK")]
vjk = pont[(pont.Kar == "VJK")]
jak = pont[(pont.Kar == "JAK")]
#cols = np.repeat(["red", "green", "blue", "orange"], 15)[0:51]
#cols

In [19]:

hist_btk, edges_btk = np.histogram(btk.Max_pontszam, density=False,
                                   range = [min(pont.Max_pontszam), max(pont.Max_pontszam)], bins = 50)
hist_itk, edges_itk = np.histogram(itk.Max_pontszam, density=False,
                                   range = [min(pont.Max_pontszam), max(pont.Max_pontszam)], bins = 50)
hist_vjk, edges_vjk = np.histogram(vjk.Max_pontszam, density=False, 
                                   range = [min(pont.Max_pontszam), max(pont.Max_pontszam)], bins = 50)
hist_jak, edges_jak = np.histogram(jak.Max_pontszam, density=False, 
                                   range = [min(pont.Max_pontszam), max(pont.Max_pontszam)], bins = 50)
hist_vjk

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int64)

In [20]:
pont.figure = figure(title = "Maximum ponthatarok")

pont.figure.quad(top=hist_btk, bottom=0, left=edges_btk[:-1], right=edges_btk[1:],
        fill_color="red", line_color="#033649", legend = "BTK")
pont.figure.quad(top=hist_itk, bottom=0, left=edges_btk[:-1], right=edges_btk[1:],
        fill_color="green", line_color="#033649", legend = "ITK")
pont.figure.quad(top=hist_jak, bottom=0, left=edges_btk[:-1], right=edges_btk[1:],
        fill_color="orange", line_color="#033649", legend = "JAK")
pont.figure.quad(top=hist_vjk, bottom=0, left=edges_btk[:-1], right=edges_btk[1:],
        fill_color="blue", line_color="#033649", legend = "VJK")
show(pont.figure)

In [44]:
hist_btk, edges_btk = np.histogram(btk.Min_pontszam, density=False,
                                   range = [min(pont.Min_pontszam), max(pont.Min_pontszam)], bins = 50)
hist_itk, edges_itk = np.histogram(itk.Min_pontszam, density=False,
                                   range = [min(pont.Min_pontszam), max(pont.Min_pontszam)], bins = 50)
hist_vjk, edges_vjk = np.histogram(vjk.Min_pontszam, density=False, 
                                   range = [min(pont.Min_pontszam), max(pont.Min_pontszam)], bins = 50)
hist_jak, edges_jak = np.histogram(jak.Min_pontszam, density=False, 
                                   range = [min(pont.Min_pontszam), max(pont.Min_pontszam)], bins = 50)
pont.figure = figure(title = "Minimum ponthatarok")

pont.figure.quad(top=hist_btk, bottom=0, left=edges_btk[:-1], right=edges_btk[1:],
        fill_color="red", line_color="#033649", legend = "BTK")
pont.figure.quad(top=hist_itk, bottom=0, left=edges_btk[:-1], right=edges_btk[1:],
        fill_color="green", line_color="#033649", legend = "ITK")
pont.figure.quad(top=hist_jak, bottom=0, left=edges_btk[:-1], right=edges_btk[1:],
        fill_color="orange", line_color="#033649", legend = "JAK")
pont.figure.quad(top=hist_vjk, bottom=0, left=edges_btk[:-1], right=edges_btk[1:],
        fill_color="blue", line_color="#033649", legend = "VJK")
show(pont.figure)

In [54]:
p = figure(title = "Ponthatar evolucio btk",plot_width=350, plot_height=350)
p.scatter(btk.Min_pontszam, btk.Max_pontszam,legend = "BTK", color = "red")
p.line(btk.Min_pontszam, btk.Max_pontszam,legend = "BTK", color = "red")
p2 = figure(title = "Ponthatar evolucio itk",plot_width=350, plot_height=350)
p2.scatter(itk.Min_pontszam, itk.Max_pontszam,legend = "ITK", color = "green")
p2.line(itk.Min_pontszam, itk.Max_pontszam,legend = "ITK", color = "green")
p3 = figure(title = "Ponthatar evolucio ajk",plot_width=350, plot_height=350)
p3.scatter(jak.Min_pontszam, jak.Max_pontszam,legend = "JAK", color = "orange")
p3.line(jak.Min_pontszam, jak.Max_pontszam,legend = "JAK", color = "orange")
p4 = figure(title = "Ponthatar evolucio vjk",plot_width=350, plot_height=350)
p4.scatter(vjk.Min_pontszam, vjk.Max_pontszam,legend = "VJK", color = "blue")
p4.line(vjk.Min_pontszam, vjk.Max_pontszam,legend = "VJK", color = "blue")
s = gridplot([[p,p2], [p3, p4]])
show(s)


In [42]:
## joining the tables
joined = pd.merge(allamiak, pont, on = ["Ev", "Kar"])

,Kar,Ev,Tipus,Ossz_Jel,Elso_Jel,Nappali_Jel,Allami_Jel,Ossz_Felv,Nappali_Felv,Allami_Felv,Max_pontszam,Min_pontszam,Max_pont
0,BTK,2015,A,5713,1781,4028,5365,1332,857,1185,0.936000,0.560000,500
1,BTK,2014,A,4386,1314,2803,4117,1240,749,1123,0.930000,0.530000,500
2,BTK,2013,A,3149,907,2111,2855,882,609,782,0.930000,0.512000,500
3,BTK,2012,A,4440,1195,3078,3856,1031,627,693,0.912500,0.754167,480
4,BTK,2011,A,3846,1192,3004,3378,1081,732,647,0.902083,0.764583,480
5,BTK,2010,A,3752,1117,3054,3395,1013,744,628,0.929167,0.735417,480
6,BTK,2009,A,3278,926,2594,2895,961,699,627,0.916667,0.662500,480
7,BTK,2008,A,2704,668,2452,2345,707,606,411,0.947917,0.445833,480
8,BTK,2007,A,3115,832,2838,2654,752,635,443,0.958333,0.826389,144
9,BTK,2006,A,4531,1283,4057,3977,977,761,542,0.965278,0.750000,144


In [55]:
joined = ColumnDataSource(joined)

In [58]:
joined.add(np.repeat(["red", "green", "orange", "blue"], 15)[0:53], name="colors")

'colors'

In [66]:
p = figure(title = "Felvettek szama es ponthatarok",plot_width=550, plot_height=550)
p.scatter("Min_pontszam","Allami_Felv",  source = joined, color = "colors")
show(p)

In [67]:
p = figure(title = "Felvettek szama es ponthatarok",plot_width=550, plot_height=550)
p.scatter("Max_pontszam","Allami_Felv",  source = joined, color = "colors")
gp.
show(p)